In [1]:
import pandas as pd
import numpy as np

meta=pd.read_csv('indexed_meta.csv', header=0, index_col=0)
meta.head()

,index,Sample,Project,SampleName,PMID,Genotype,Ecotype,Tissue,TotalReads,UniqueMappedRate,ReleaseDate
0,1,DRX007662,PRJDB2180,Arabidopsis WT-Col mRNA_seq,23934508,wild type,Col-0,/,30664389,86.20%,4/2/2014
1,2,DRX007663,PRJDB2180,Arabidopsis ibm1-4 mRNA_seq,23934508,ibm1-4,Col-0,/,38551905,91.10%,4/2/2014
2,3,DRX007664,PRJDB2180,Arabidopsis ibm2-2 mRNA_seq,23934508,ibm2-2,Col-0,/,37223057,83.40%,4/2/2014
3,4,DRX014481,PRJDB1593,Y1,/,wild type,/,root,95012910,89.80%,2/5/2016
4,5,DRX014482,PRJDB1593,Y2,/,wild type,/,root,163269003,92.90%,2/5/2016


In [ ]:
### R didn't mind the original formatting but python's a lil less forgiving

In [3]:
meta['UniqueMappedRate'] = meta['UniqueMappedRate'].str.rstrip('%').astype('float') #UMR was held as string with % appended so get rid of that & convert to float
meta.head()

,index,Sample,Project,SampleName,PMID,Genotype,Ecotype,Tissue,TotalReads,UniqueMappedRate,ReleaseDate
0,1,DRX007662,PRJDB2180,Arabidopsis WT-Col mRNA_seq,23934508,wild type,Col-0,/,30664389,86.2,4/2/2014
1,2,DRX007663,PRJDB2180,Arabidopsis ibm1-4 mRNA_seq,23934508,ibm1-4,Col-0,/,38551905,91.1,4/2/2014
2,3,DRX007664,PRJDB2180,Arabidopsis ibm2-2 mRNA_seq,23934508,ibm2-2,Col-0,/,37223057,83.4,4/2/2014
3,4,DRX014481,PRJDB1593,Y1,/,wild type,/,root,95012910,89.8,2/5/2016
4,5,DRX014482,PRJDB1593,Y2,/,wild type,/,root,163269003,92.9,2/5/2016


In [ ]:
### cut that mess down a little

In [ ]:
### by unique mapped rate, here > 90% bc i wanted smaller data to play with locally
### just change umr_thresh to whatever your preferred cutoff is
### >90% drops

### keeping this cut across all subsequent filters, but doing the rest seperately & compiling at end

umr_thresh=90
meta_by_umr=meta[meta["UniqueMappedRate"] > umr_thresh]
meta_by_umr.head()

In [127]:
#just for reference, how many samples made it past that cut?
complete=len(meta)
by_umr=len(meta_by_umr)
print(complete, "samples reduced to", by_umr, "by filtering unique mapped read rate\nTotal number of samples reduced by ",((1-(by_umr/complete))*100),"%.")

28164 samples reduced to 6930 by filtering unique mapped read rate
Total number of samples reduced by  75.3941201533873 %.


In [ ]:
### by ecotype

### example is just Col-0 & some others with low counts bc i wanted a smaller data set.. & to show filtering by multiple ecotypes at the same time

In [ ]:
### first let's see what options we have

ecotypes=meta["Ecotype"].value_counts() # unique ecotype entries in the full meta & how many are named this
ecotypes=ecotypes.to_frame()
ecotypes.head()  #note that this has 1529 entries..

In [129]:
### now subset. 
### i'm using str.fullmatch() for exact str matching, but could use str.match() or str.contains() for wider search
### remember that this is coded for data already filtered by UMR while the list above was not
### some strings will not return results here unless you change it to read from the full meta

#be careful with this if selecting weirdly named ones. you might need to do them seperately & merge your results..

eco_to_keep="Col-0|Col-4|Landsberg erecta" #set strings to include, use as many or few as you want. just put | operators btwn strings without spaces like shown
meta_by_eco=meta_by_umr[meta_by_umr["Ecotype"].astype(str).str.fullmatch(eco_to_keep)] 
meta_by_eco["Ecotype"].value_counts() #print what's in the resulting df to check that it worked

Col-0               5880
Col-4                 32
Landsberg erecta      11
Name: Ecotype, dtype: int64

In [109]:
### samples left
by_eco=len(meta_by_eco)
print(by_umr, "samples reduced to", by_eco, "after filtering by ecotype\nnumber of samples reduced by ",((1-(by_eco/by_umr))*100),"%.")

6930 samples reduced to 5923 after filtering by ecotype
number of samples reduced by  14.531024531024528 %.


In [ ]:
### by genotype

### example is just wild type(s) bc it's essentially the same as by ecotype

In [128]:
###check the options

genotypes=meta["Genotype"].value_counts() # unique genotype entries in the full meta & how many are named this
genotypes=genotypes.to_frame()
genotypes.head()  #note that this has 2076 entries 

In [59]:
### subset
geno_to_keep="wild type|Wild type|wild-type|Wild Type" #set strings to include, use as many or few as you want. just put | operators btwn strings without spaces like shown
meta_by_geno=meta_by_umr[meta_by_umr["Genotype"].astype(str).str.fullmatch(geno_to_keep)] 
meta_by_geno["Genotype"].value_counts() #print what's in the resulting df to check that it worked

wild type    1498
Wild type     103
Wild Type      10
wild-type       9
Name: Genotype, dtype: int64

In [110]:
### samples left
by_geno=len(meta_by_geno)
print(by_umr, "samples reduced to", by_geno, "after filtering by genotype\nnumber of samples reduced by ",((1-(by_geno/by_umr))*100),"%.")

6930 samples reduced to 1620 after filtering by genotype
number of samples reduced by  76.62337662337663 %.


In [ ]:
### by tissue

### example is just root bc it's basically the same again

In [130]:
###check the options

tissues=meta["Tissue"].value_counts() # unique tissue entries in the full meta & how many are named this
tissues=tissues.to_frame()
tissues.head()  #note that this has 333 entries

In [63]:
### doing this a little different bc variations in naming are more likely to be unintentional here
### previous examples are written using str.fullmatch() but changing that to str.contains()
### returns all versions that contain string, here 'root' as example

tissue_to_keep="root" #set strings to include, use as many or few as you want. just put | operators btwn strings without spaces like shown
meta_by_tissue=meta_by_umr[meta_by_umr["Tissue"].astype(str).str.contains(tissue_to_keep)] 
meta_by_tissue["Tissue"].value_counts() #print what's in the resulting df to check that it worked

root                   461
root tip                37
roots                   36
root tips               15
seedling root            7
root QC cells            7
root tissue              6
whole root               6
root and leaf            6
7-day-old root tips      4
root hair                3
leaf, stem, root         3
10 day old roots         1
1cm long root tips       1
Name: Tissue, dtype: int64

In [112]:
### samples left
by_tissue=len(meta_by_tissue)
print(by_umr, "samples reduced to", by_tissue, "after filtering by tissue\nnumber of samples reduced by ",((1-(by_tissue/by_umr))*100),"%.")

6930 samples reduced to 593 after filtering by tissue
number of samples reduced by  91.44300144300144 %.


In [131]:
### combine filters
### i would personally just take each subset from the output of the one before it since this would use less computational resources
### but for the sake of demonstration i took each subset from the whole UMR filtered set

meta_eco_geno=meta_by_eco.merge(meta_by_geno, how='inner') # merge result is only those shared between the two df
meta_eco_geno_tissue=meta_eco_geno.merge(meta_by_tissue, how='inner')

### samples left
by_ecoxgeno=len(meta_eco_geno)
by_all=len(meta_eco_geno_tissue)
print(by_umr, "samples reduced to", by_all, "after filtering by ecotype, genotype, & tissue\nfiltering by ecotype removed", (by_umr-by_eco), "samples\nfiltering by genotype removed an additional", (by_eco-by_geno), "samples\nfiltering by tissue removed an additional", (by_eco-by_geno-by_ecoxgeno), "samples\nnumber of samples reduced by ",((1-(by_all/by_umr))*100),"%.")
meta_eco_geno_tissue.head(20)

6930 samples reduced to 113 after filtering by ecotype, genotype, & tissue
filtering by ecotype removed 1007 samples
filtering by genotype removed an additional 4303 samples
filtering by tissue removed an additional 2869 samples
number of samples reduced by  98.36940836940838 %.


In [154]:
### almost there, yay!
### now we need to be able to get those samples pulled from the data on hpcc
### let's see if we can get the whole bash command printed for an easy copypasta

subset=meta_eco_geno_tissue #chonky name is annoying

subset_index=subset['index'] # list the index numbers of subset
subindx2=subset_index+1 # +1 bc awk starts at 1 & first column is geneIDs
indx=subindx2.astype(str).tolist() #make it a list of strings so we can format it how awk likes
awk_index = ["$"+ n for n in indx] #add $ to index for awk (i think this tells it to pick columns? not sure tbh)

print("awk -F, \'{OFS=\",\";print $1,", ', '.join(awk_index), "}\' gene_FPKM_200501.csv > subset.csv") # print the whole bash cmd

### & there's your bash cmd that should be able to run on a dev node unless your final subset is really big
### probably should either run this on a copy of the full data in your home directory on hpcc or specify full output path
### just make sure to replace subset.csv with something more meaningful

awk -F, '{OFS=",";print $1, $160, $161, $162, $1038, $2521, $3910, $3911, $3914, $3915, $3918, $3919, $3922, $3926, $3930, $3931, $3936, $4481, $7952, $7953, $9095, $9097, $9101, $9102, $9103, $9264, $10484, $10499, $10500, $13541, $14342, $14344, $14345, $15371, $15372, $15373, $15374, $15376, $15646, $15647, $15648, $15649, $17000, $17001, $17004, $17005, $17696, $17697, $17698, $17702, $17703, $17704, $17954, $17955, $20746, $20754, $20755, $20758, $20759, $22045, $22046, $22047, $22991, $22992, $22994, $23235, $23236, $23237, $23238, $23239, $23240, $23241, $23242, $23243, $23273, $24846, $24847, $25028, $25040, $25041, $25042, $25043, $25044, $25121, $25122, $25123, $25265, $25267, $25269, $25270, $27601, $27602, $27774, $27775, $27776, $27984, $27985, $27994, $27995, $27996, $27997, $28016, $28017, $28018, $28019, $28034, $28035, $28036, $28039, $28040, $28041, $28044, $28045, $28046 }' gene_FPKM_200501.csv > subset.csv


In [ ]:
### hopefully someone gets some use out of this, it's a lot cleaner & more informative than the initial R version i wrote
### i'll also share a cut down .py version of this on github so that it's easier to run (just make sure to adjust parameters, use the indexed metadata, & adjust your output file name)

### parameters that should definitely be adjusted: umr_thresh, eco_to_keep, geno_to_keep, tissue_to_keep
### additional adjustments: str.fullmatch() vs str.match() vs str.contains() when filtering by category
###                         how (or if) you merge results.. merging all 3 filters is shown but could do any two, or could skip that step via serial filtering

### the code to extract the example subset (113 of the original 28164 samples) took 1m38s on a dev-amd20 node
### not sure how this scales, but subsets up to ~5k samples should be safe to run on a dev node
### depending on workflow it might be a good idea to follow this up with a modified version of my filter.py script to remove genes with consistently low expression.. just make sure to bypass the log2 transformation